### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Summer 2018 ###

# Factorio Optimization #

#### Robert Claus (rclaus@wisc.edu), Jackson Lanigan (jlanigan@wisc.edu)

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
    1. [Mineral Rich Mining](###4.A.-Mineral-Rich-Mining)
    1. [Gaussian Results](###4.B.-Gaussian-probability-distribution)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

The computer game Factorio involves building a factory to collect resources, process them, and combine them into other objects. In the same vein as other games such as Minecraft, the goal is to ramp up the complexity of what you are combining to reach higher tiers of objects. A key difference between Factorio and other games is its focus on automating the process rather than collecting resources. A good amount of the game is dedicated to optimizing your factories and layouts to produce resources as efficiently as possible.

As an example, say you decided on a 3x3 grid for your factory you wish to use optimally. You have mining machines, smelters, conveyor belts to transport resources between machines and a chest you wish to place the smelted metal in.  You know which spaces in the grid contain metals to mine and which do not.  Given that you know how fast the mining machine and smelters can operate (mining machines produce more material than a smelter can handle) how do you lay out your factory to maximize the smelted material you produce?

This project models a simplified version of the Factorio game and optimize the factory layout.  Given the online community a number of projects already exist to [calculate general resource requirements](https://kirkmcdonald.github.io/calc.html#items=advanced-circuit:f:1) given a target and [example layouts](https://steamcommunity.com/sharedfiles/filedetails/?id=754378586) that scale well as your factory grows.  Our focus is instead on optimizing modular sections of the factory that come up in most designs.  


To make the problem clear, we present a simple example. For our simplified version of the game, a scenario may involve a 5 by 3 grid with resources along only the top edge of the map.  Hence our resource distribution for this scenario would be:

| | | | ||
|---| --- |--- |--- |--- |--- |
|1 |1 |1 |1 |1|
|0 |0 |0 |0 |0|
|0 |0 |0 |0 |0|

We have mining machines that we can place on the map that each take up one space and face any direction.  These feed whatever resources they mine out in the direction they face. We can expect an optimal layout on the resources above to have a mining machine on each of the spots having resources. Other mining machines do not accept resources onto their space, so the only way to get the resources they mine onto another space would be facing down.  We refer to buildings on the map by a one letter code for their type followed by a single letter indicating their direction. A mining machine would be "m" and the direction facing down as "d".  Hence a mining machine facing down would be designated "md".

Hence, we would expect the optimal placement of miners to look something the grid below.  (With . representing an empty space)

| | | | ||
|---| --- |--- |--- |--- |--- |
|md |md |md |md |md |
|. |. |. |. |.|
|. |. |. |. |.|

Total Production: 5, 
Collected Resources: 0

From here we are feeding resources from the first row into the second.  However, we want to collect the resources into a storage chest.  If we place this chest in front of one of the mining machines it will start collecting resources.  We call a chest "h" and it doesn't have a direction.  Let's put it on the map!

| | | | ||
|---| --- |--- |--- |--- |--- |
|md |md |md |md |md |
|. |. |. |. |h  |
|. |. |. |. |.|
Total Production: 5, 
Collected Resources: 1

This is great since we are now collecting resources from the right most mining machine into the chest.  However, we have four other mining machines with resources that aren't reaching the chest.  Let's build some conveyor belts to cary the resources from the mining machine to the chest.  Conveyors accept resources entering their space from any direction and will push them all to the next space in the direction they face.  We will call conveyors "c" and in this case they will face to the right and hence have a direction "r".

| | | | ||
|---| --- |--- |--- |--- |--- |
|md |md |md |md |md |
|cr |cr |cr |cr |h  |
|. |. |. |. |.|

Total Production: 5, 
Collected Resources: 5

On this map we can't do much better since there aren't any more resources.  On more complicated resource distributions many interesting factors come into play.  For instance, imagine if conveyor belts could only carry 3 resources.  This would be a problem because the rightmost conveyor needs to carry 4 resources from the 4 mining machines it connects to the chest.  To solve this, we may need to add more conveyors!

| | | | ||
|---| --- |--- |--- |--- |--- |
|md |md |md |md |md |
|cr |cr |cd |cr |h  |
|. |. |cr |cr |cu|

Total Production: 5, 
Collected Resources: 5

In this more complicated solution we route three of the resources down and back up into the chest so that no conveyor needs to carry more than 3 resources.  ("d" is facing down, "r" is facing right, and "u" is facing up.)

In the project we analyze much more complicated situations and map configurations, but they all condense into the same basic problem of placing as many miners as possible while still routing all of the resources to the chest.

Data for the project was [available online](https://wiki.factorio.com/Mining) and we explored how these parameters affect optimal layouts.

## 2. Mathematical model ##

Now that you have an introduction to our topic, we will break down the model we are using to analyze the topic. The model itself is a variation of a network flow linear program. We will begin by creating a square map using a Gaussian distribution function. To make the model as basic as possible, we found parameters we could implement to simply create a map where every space provides an equal number of resources (to make this simple, we'll just use 1 as the resource value). 

We have a decision to make for each node/space on the map. We can place nothing on a space, or we can place a miner, conveyor, or a chest. More specifically, the miners and conveyors must be given a specific direction to decide which direction they will carry the resource to. So our decision is to decide for each space on the map, whether to do nothing, place a chest, a miner (and if so, in which direction), or a conveyor (and if so, in which direction). This comes to 10 possible building options. 

To represent this process, we use a matrix the same size as the map, with each element of the matrix representing a variable at a specific spot on the map: 

1-10: A binary variable for each building type representing whether or not that building type will be used in the corresponding space on the map.

11: A variable for chests, representing how many resources flow into a chest at the corresponding location on the map.

12-15: A variable for each direction of miner, representing how many resources will be mined at the corresponding location on the map.

16-19: A variable for each direction of conveyor, representing how many resources will be transferred at the corresponding location on the map.

In order to make this model work in the way that we would like, we have many constraints we must implement.

1: Balance constraint: The flow out must equal the flow in at each node for a network flow problem.

2: We must limit the chest contents to zero for each space that is not a chest.

3-6: A miner has a limit on production in each direction.

7-10: A conveyor has a capacity limit in each direction.

11-14: We must create a buffer at each edge of the map to ensure there is no flow from a space on the map to a space          off the map.

15: We constrain the maximum number of buildings of each type. These values come from a parameter introduced before       the variables and constraints are defined.

16: There can only be one building in each space.

Our objective is to maximize the total flow of resources into chests. Another factor we include in the objective function is the cost, which is insignificant to us in the context of our problem, but will hold a very slight weight in our analysis. The reason for this inclusion of cost in the objective is to ensure that if we do not increase flow into a chest, then we will simply do nothing with a space. For example, if we have already maximized the capacity of conveyors surrounding a chest with spots to fill, we would simply do nothing with the remaining spaces. We can ensure the model performs this step correctly by introducing a cost for each machine, while assigning no cost to the option of "do nothing". Cost could potentially be a factor to explore in the future, with a Pareto tradeoff analysis in which we change the parameter to increase the weight that cost holds in the objective function.

Finally, there are many parameters we introduce in this model in order to do a proper sensitivity analysis and to organize the model in a presentable format. We must first build the resource distribution map before introducing the variables, and introduce some parameters that affect different factors of the model. We will explore the impact that changing some of these parameters can have on the overall result.

Here is the model written out in standard form. The model is more intuitive with some of the variables on the right hand side of the constraints, especially the balance constraint, which you can see in the code. One important note is that there are a few parameters used in the constraints. We can assume for our purposes that these are values that have already been defined, which validates our model. In our actual model, we will define these parameter values before running through the model. 

|Parameters|
|---|---|
|building_cost[i]        |
|destination_capacity[i] |
|resources[x,y]          |
|nothing_index           |
|max_count[i]            |



$$\begin{aligned}
  \underset{ \mathbb{}}{\text{maximize}}\qquad& \sum_{y \in map\_indexes} \sum_{x \in map\_indexes}(dest[x,y]) \space - \sum_{y \in map\_indexes} \sum_{x \in map\_indexes} \sum_{i=1}^{building\_count}(type[i][x,y]*building\_cost[i])\\
    \text{subject to:}\qquad& -(prod_r[x-1,y]+prod_d[x,y-1]+prod_u[x,y+1]+prod_l[x+1,y]+pass_r[x-1,y]+pass_d[x,y-1]+pass_u[x,y+1]+pass_l[x+1,y]-dest[x,y]-pass_r[x,y]-pass_d[x,y]-pass_i[x,y]-pass_l[x,y]) \le 0 && \forall \space x,y \in map\_indexes\_inside\\
    & dest[x,y] - \sum_{i=1}^{building\_count}(destination\_capacity[i]*type[i][x,y]) \le 0 && \forall \space x,y \in map\_indexes\_inside\\
    & prod_r[x,y] - \sum_{i=1}^{building\_count}(production\_right[i]*resources[x,y]*type[i][x,y]) \le 0 && \forall \space x,y \in map\_indexes\_inside\\
    & prod_d[x,y] - \sum_{i=1}^{building\_count}(production\_down[i]*resources[x,y]*type[i][x,y]) \le 0 && \forall \space x,y \in map\_indexes\_inside\\
    & prod_u[x,y] - \sum_{i=1}^{building\_count}(production\_up[i]*resources[x,y]*type[i][x,y]) \le 0 && \forall \space x,y \in map\_indexes\_inside\\
    & prod_l[x,y] - \sum_{i=1}^{building\_count}(production\_left[i]*resources[x,y]*type[i][x,y]) \le 0 && \forall \space x,y \in map\_indexes\_inside\\
    & pass_r[x,y] - \sum_{i=1}^{building\_count}(max\_passthrough\_right[i]*type[i][x,y]) \le 0 && \forall \space x,y \in map\_indexes\_inside\\
    & pass_d[x,y] - \sum_{i=1}^{building\_count}(max\_passthrough\_down[i]*type[i][x,y]) \le 0 && \forall \space x,y \in map\_indexes\_inside\\
    & pass_u[x,y] - \sum_{i=1}^{building\_count}(max\_passthrough\_up[i]*type[i][x,y]) \le 0 && \forall \space x,y \in map\_indexes\_inside\\
    & pass_l[x,y] - \sum_{i=1}^{building\_count}(max\_passthrough\_left[i]*type[i][x,y]) \le 0 && \forall \space x,y \in map\_indexes\_inside\\
    & type[nothing\_index][map\_size,x] \le 1 && \forall \space x \in map\_indexes\\ \\
    & -type[nothing\_index][map\_size,x] \le -1 && \forall \space x \in map\_indexes\\ \\
    & type[nothing\_index][x,map\_size] \le 1 && \forall \space x \in map\_indexes\\ \\
    & -type[nothing\_index][x,map\_size] \le -1 && \forall \space x \in map\_indexes\\ \\
    & type[nothing\_index][x,1] \le 1 && \forall \space x \in map\_indexes\\ \\
    & -type[nothing\_index][x,1] \le -1 && \forall \space x \in map\_indexes\\ \\
    & type[nothing\_index][1,x] \le 1 && \forall \space x \in map\_indexes\\ \\
    & -type[nothing\_index][1,x] \le -1 && \forall \space x \in map\_indexes\\ \\
    & \sum_{x,y \in map\_indexes}(type[i][x,y]) - max\_count[i] \le 0.1 && \forall \space i=1,\dots,building\_count\\
    & \sum_{i=1}^{building\_count}(type[i][x,y]) \le 1 && \forall \space x,y \in map\_indexes\_inside\\
    & -\sum_{i=1}^{building\_count}(type[i][x,y]) \le -1 && \forall \space x,y \in map\_indexes\_inside\\
    & type[x,y] \ge 0 && \forall \space x,y \in map\_indexes\\
    & dest[x,y] \ge 0 && \forall \space x,y \in map\_indexes\\
    & prod_r[x,y] \ge 0 && \forall \space x,y \in map\_indexes\\
    & prod_d[x,y] \ge 0 && \forall \space x,y \in map\_indexes\\
    & prod_u[x,y] \ge 0 && \forall \space x,y \in map\_indexes\\
    & prod_l[x,y] \ge 0 && \forall \space x,y \in map\_indexes\\
    & pass_r[x,y] \ge 0 && \forall \space x,y \in map\_indexes\\
    & pass_d[x,y] \ge 0 && \forall \space x,y \in map\_indexes\\
    & pass_u[x,y] \ge 0 && \forall \space x,y \in map\_indexes\\
    & pass_l[x,y] \ge 0 && \forall \space x,y \in map\_indexes\\
    \end{aligned}$$

Here are the variable names we use in standard form (left column) and how they're represented in our code (right column). Everything else in standard form has the same name:

|||
|-----|-------------------------|
|dest    | var\_destination\_contents |
|prod_r  | var\_production\_right     |
|prod_d  | var\_production\_down      |
|prod_u  | var\_production\_up        |
|prod_l  | var\_production\_left      |
|pass_r  | var_passthrough_right      |
|pass_d  | var_passthrough_down       |
|pass_u  | var_passthrough_up         |
|pass_l  | var_passthrough_left       |

## 3. Solution ##

The first thing we will do in our model is introduce parameters and set up the map. There are many scenarios that we will run through to get an understanding of how sensitive our model is to the parameters we use. There are many possible scenarios to run through involving the size and distribution of the map as well as the characteristics of the buildings, but we will choose some of the scenarios we found to be most interesting and insightful. We will isolate each of these cases to display the clear effect that each of these specific demonstrations has, though we found that there are many combinations of changes in parameters that would be interesting for future analysis.

We use various map sizes depending on the scenario we intend to evaluate. We start with a map size of three for the basic scenario. The miner speed and conveyor speed represent the maximum capacity that each direction of miner and conveyor can operate at relative to the resource amount. As a result, conveyors can carry a maximum of six total resources at one time, given that a miner mines one resource in the same amount of time. We use an arbitrarily large number for maximum chest contents, given that we would like to maximize the number of resources that flow into a chest. We will normally use one chest for each scenario, but will also evaluate the impact of increasing the number of chests in our results and discussion.

We introduce a parameter that we will evaluate in which we throw noise into the probability distribution of the map. The next parameter is one that normalizes the distribution so that with all the other parameters, the map will simply be uniform, with one resource in each entry. We will eventually change this to introduce a true Gaussian probability distribution.

Finally, we have a parameter for the standard deviation and offset in the x and y directions, allowing us to move around the resource layout of the map.

In [73]:
# We begin with the parameters
map_size = 4
miner_speed = 1
conveyor_speed = 6
max_chest_contents = 100
number_of_chests = 1

resource_noise = 0
resource_scale = 63000

x_standard_deviation = 100
y_standard_deviation = 100

x_offset = 0
y_offset = 0

# Introduce buffer region around edges of map so that we can later ensure that resources will not flow off the map
map_size = map_size + 2

# Calculate the middle of the map as a parameter to offset when we want the mean somewhere other than the middle
middle = (map_size + 1)/2

# Define mean and standard deviation for x and y
x_mean = middle + x_offset   # We can adjust the mean of x or y to shift the distribution from the center
y_mean = middle + y_offset
x_sd = x_standard_deviation  # We can adjust the standard deviation to expand or compress distribution from the center
y_sd = y_standard_deviation

# Define noise
param = resource_noise   # Maximum change in percentage of a resource in each space; implemented below
param2 = resource_scale  # This adjusts the resource distribution to better fit our model

# Initiate empty square matrix with length of map size
resources = zeros(map_size, map_size);

After introducing the parameters, we now build the map. The noise is implemented using a random function, where we multiply the probability of each entry by a percentage between (1-noise parameter) and (1+noise parameter), thus increasing or decreasing the value by a random percentage in the given range.

In [74]:
# *****BUILD RESOURCE DISTRIBUTION*****
for x = 1:map_size
    for y = 1:map_size
        # Noise implementation
        noise = rand((1-param):0.01:(1+param))
        # Gaussian distribution with noise
        resources[x,y] = param2 * noise * (e^(-1/2 * (((x - x_mean)/x_sd)^2 + ((y - y_mean)/y_sd)^2)) / (2*pi * x_sd * y_sd))
    end
end

##########

# Need a value to set an upper limit on resource production
resource_max = maximum(resources[x,y] for x in 1:map_size, y in 1:map_size)
max_passthrough = conveyor_speed
max_capacity = max_chest_contents;

Now the resource map is established, which means we can begin setting up the actual model. We create important vectors that will be used as parameters when creating the variables and constraints. It is important to note what the abbreviations for each possible building are:

|||
| --- | --- |
|.  | nothing        |
|h  | chest          |
|ml | miner left     |
|mr | miner right    |
|mu | miner up       |
|md | miner down     |
|cl | conveyor left  |
|cr | conveyor right |
|cu | conveyor up    |
|cd | conveyor down  |

We will introduce the cost for each option, which is especially significant when we write the objective function. When the model runs, if is not beneficial to create a building in a certain space (in other words, if building a chest, conveyor, or miner does not add to our objective), then the model should decide to do nothing. 

In [75]:
# ****************************************************** MODEL ******************************************************

using JuMP, Gurobi
m = Model(solver=GurobiSolver(Presolve=-1, OutputFlag=0))

map_indexes_inside = 2:map_size-1   # Doesn't include edges of map
map_indexes = 1:map_size            # Includes edges of map; important to prevent flow from leaving the map

# Building Configuration
nothing_index = 1       # Do nothing
chest_index = 2         # Build a chest
building_count = 10     # Possible building options include nothing, chest, and miners and conveyors in all directions

# Building Configuration

building_letter =       [".  ","h  ", "mr ", "md ","mu ","ml ","cr ", "cd ", "cu ", "cl "]

# Can change cost of each building to encourage certain buildings or directions over others. Very important to include
# cost in the objective because it will allow the model to default to "nothing" if a building does not improve flow
# into chest.

building_cost =         [0,  1,  1,  1.5,  1.75,  2,  1,  1.5,  1.75,  2]

# Create a vector for chest and each direction of miner and conveyor to limit capacity
destination_capacity =  [0,  max_chest_contents,  0,  0,  0,  0,  0,  0,  0,  0]
production_right =      [0,  0,  miner_speed,  0,  0,  0,  0,  0,  0,  0]
production_down =       [0,  0,  0,  miner_speed,  0,  0,  0,  0,  0,  0]
production_up =         [0,  0,  0,  0,  miner_speed,  0,  0,  0,  0,  0]
production_left =       [0,  0,  0,  0,  0,  miner_speed,  0,  0,  0,  0]
max_passthrough_right = [0,  0,  0,  0,  0,  0,  conveyor_speed,  0,  0,  0]   # The indices are very important
max_passthrough_down =  [0,  0,  0,  0,  0,  0,  0,  conveyor_speed,  0,  0]
max_passthrough_up =    [0,  0,  0,  0,  0,  0,  0,  0,  conveyor_speed,  0]
max_passthrough_left =  [0,  0,  0,  0,  0,  0,  0,  0,  0,  conveyor_speed]

# Max of each building option should be arbitrarily large except for the chest, which is a parameter we introduced
max_count =             [50,number_of_chests,50,50,50,50,50,50,50,50];

Now that these vectors are established we are able to define the decision variables. Essentially, the overall decision we must make is as follows: given the resource map we are provided, what should we build in each space available on the map in order to maximize the number of resources inside a chest when the model is finished? The building options are to do nothing, to build a chest(s), to build a miner that will feed the resources in a certain direction, or to build a conveyor that will push flow in a certain direction. We will use a dictionary to build these variables.

In [76]:
# VARIABLES

# Each variable will be defined using Dict()
# We create a map_size x map_size binary variable matrix for all 10 building options
var_is_a = Dict()
for i in 1:building_count
    # Each entry of the array is a binary variable representing whether or not we will use that type of building.
    var_is_a[i]=
        @variable(m, 
            [1:map_size, 1:map_size], 
            lowerbound = 0, 
            upperbound = 1, 
            Bin
            )
end

# map_size x map_size matrix with each entry as a chest variable
# Each entry will be zero if not a chest, will be between zero and max_capacity parameter if it is a chest
var_destination_contents =
    @variable(m, [1:map_size,1:map_size], 
        lowerbound=0, 
        upperbound=max_capacity
        )

# For the next 4 variables:
# map_size x map_size matrix with each entry as a miner variable for the corresponding direction
# Each entry will be zero if not a miner in the given direction, between zero and resource_max if it is

var_production_right=
    @variable(m, 
        [1:map_size,1:map_size], 
        lowerbound = 0, 
        upperbound = resource_max
        )

var_production_down=
    @variable(m, 
        [1:map_size,1:map_size], 
        lowerbound = 0, 
        upperbound = resource_max
        )

var_production_up=
    @variable(m, 
        [1:map_size,1:map_size], 
        lowerbound = 0, 
        upperbound = resource_max
        )

var_production_left=
    @variable(m, 
        [1:map_size,1:map_size], 
        lowerbound = 0, 
        upperbound = resource_max
        )

# For the next 4 variables:
# map_size x map_size matrix with each entry as a conveyor variable for the corresponding direction
# Each entry will be zero if not a conveyor in the given direction, between zero and max_passthrough if it is

var_passthrough_right=
    @variable(m, 
        [1:map_size,1:map_size], 
        lowerbound = 0, 
        upperbound = max_passthrough
        )

var_passthrough_down=
    @variable(m, 
        [1:map_size,1:map_size], 
        lowerbound = 0, 
        upperbound = max_passthrough
        )

var_passthrough_up=
    @variable(m, 
        [1:map_size,1:map_size], 
        lowerbound = 0, 
        upperbound = max_passthrough
        )

var_passthrough_left=
    @variable(m, 
        [1:map_size,1:map_size], 
        lowerbound = 0, 
        upperbound = max_passthrough
        );

Now that the variables are established, we can build the constraints and run the objective function.

In [77]:
# Flow balance constraint
# Runs through each entry; takes sum of resources being produced or passed in entries in any of the four directions
# from the entry, which must equal the sum of everything passing through or being produced within the entry itself

@constraint(m, flow_left[x in map_indexes_inside,y in map_indexes_inside],
        var_production_right[x-1,y] +
        var_production_down[x,y-1] +
        var_production_up[x,y+1] +
        var_production_left[x+1,y] +  
        
        var_passthrough_right[x-1,y] +
        var_passthrough_down[x,y-1] +
        var_passthrough_up[x,y+1] +
        var_passthrough_left[x+1,y] 
    == 
        var_destination_contents[x,y] + 
        
        var_passthrough_right[x,y] +
        var_passthrough_down[x,y] +
        var_passthrough_up[x,y] +
        var_passthrough_left[x,y]
)

# Chest contents are 0 if not a chest
@constraint(m, chest_capacity[x in map_indexes, y in map_indexes],
    var_destination_contents[x,y]
    <= sum(destination_capacity[i]*var_is_a[i][x,y] for i in 1:building_count)
)

# If an entry is a miner, it can produce no more than the maximum miner capacity
@constraint(m, max_production_right[x in map_indexes, y in map_indexes],
    var_production_right[x,y] <=
    sum(production_right[i]*resources[x,y]*var_is_a[i][x,y] for i in 1:building_count)
)

@constraint(m, max_production_down[x in map_indexes, y in map_indexes],
    var_production_down[x,y] <=
    sum(production_down[i]*resources[x,y]*var_is_a[i][x,y] for i in 1:building_count)
)

@constraint(m, max_production_up[x in map_indexes, y in map_indexes],
    var_production_up[x,y] <=
    sum(production_up[i]*resources[x,y]*var_is_a[i][x,y] for i in 1:building_count)
)

@constraint(m, max_production_left[x in map_indexes, y in map_indexes],
    var_production_left[x,y] <=
    sum(production_left[i]*resources[x,y]*var_is_a[i][x,y] for i in 1:building_count)
)

# Integer constraints representing that if an entry is a conveyor, it can transport no more than the maximum conveyor
# capacity that we set as a parameter earlier

@constraint(m, max_passthrough_right[x in map_indexes, y in map_indexes],
    var_passthrough_right[x,y] <= 
    sum(max_passthrough_right[i]*var_is_a[i][x,y]for i in 1:building_count)
)

@constraint(m, max_passthrough_down[x in map_indexes, y in map_indexes],
    var_passthrough_down[x,y] <= 
    sum(max_passthrough_down[i]*var_is_a[i][x,y]for i in 1:building_count)
)

@constraint(m, max_passthrough_up[x in map_indexes, y in map_indexes],
    var_passthrough_up[x,y] <= 
    sum(max_passthrough_up[i]*var_is_a[i][x,y] for i in 1:building_count)
)

@constraint(m, max_passthrough_left[x in map_indexes, y in map_indexes],
    var_passthrough_left[x,y] <= 
    sum(max_passthrough_left[i]*var_is_a[i][x,y]for i in 1:building_count)
)


# Buffer edge of map by not allowing building there
@constraint(m, buffer_right[x in map_indexes],
    var_is_a[nothing_index][map_size,x]==1)
@constraint(m, buffer_bottom[x in map_indexes],
    var_is_a[nothing_index][x,map_size]==1)
@constraint(m, buffer_top[x in map_indexes],
    var_is_a[nothing_index][x,1]==1)
@constraint(m, buffer_left[x in map_indexes],
    var_is_a[nothing_index][1,x]==1)


# Maximum buildings of each type defined in max_count
@constraint(m, maximums[i in 1:building_count],
   sum(var_is_a[i][x,y] for x in map_indexes, y in map_indexes) <= max_count[i] + .1
)

# Only assign one building per space
@constraint(m, single_assignment[x in map_indexes, y in map_indexes], 
    sum(var_is_a[i][x,y] for i in 1:building_count) == 1
)

# OBJECTIVE

# Maximize resources into chests, but prefer cheaper buildings
@objective(m, Max, 
    # Amount feeding into the chests
    sum(sum(var_destination_contents[x,y] for x in map_indexes) for y in map_indexes)
    
    # Building cost of each building (to prefer nothing if building doesn't produce something)
    - 0.0001*sum(sum(
            sum(var_is_a[i][x,y]*building_cost[i] for i in 1:building_count)
        for x in map_indexes)  for y in map_indexes)
);

We would like to print these results in a neat manner. We first create four different matrices, each of which we will end up printing with different information. The first will print which building we will choose for each space (or nothing). Next will be the chest matrix, which will tell us how many resources are accumulated in each space. If we take the sum of the entries of this matrix, we have our objective value (how many total resources we were able to store in a chest). The third will be the miner matrix, which will tell us how many resources each spot produces (in other words, which entries have a miner, and what capacity they are operating at). Finally, we will do the same for conveyors. We will also print the resource distribution matrix, which will be especially important when we introduce a random parameter, such as how we have defined noise.

In [78]:
status = solve(m)
println("Calculation Complete")

# Print Results
assignment = [".  " for r in 1:map_size, c in 1:map_size]
contents = [0.0 for r in 1:map_size, c in 1:map_size]
prod = [0.0 for r in 1:map_size, c in 1:map_size]
passthrough = [0.0 for r in 1:map_size, c in 1:map_size]

for building_type in 1:building_count
    for idx = find(getvalue(var_is_a[building_type]) .> 0.9)
        assignment[idx] = building_letter[building_type]
        contents[idx] += getvalue(var_destination_contents)[idx]
        prod[idx] += getvalue(var_production_right)[idx]
        prod[idx] += getvalue(var_production_down)[idx]
        prod[idx] += getvalue(var_production_up)[idx]
        prod[idx] += getvalue(var_production_left)[idx]
        passthrough[idx] += getvalue(var_passthrough_right)[idx]
        passthrough[idx] += getvalue(var_passthrough_down)[idx]
        passthrough[idx] += getvalue(var_passthrough_up)[idx]
        passthrough[idx] += getvalue(var_passthrough_left)[idx]
    end
end

# Will print what we want to build on each space
println()
println("Assignment")
for i in 2:map_size-1
    for j in 2:map_size-1
        print(assignment[j,i]," | ")
    end
    println()
end

# Will print how many resources are placed in a chest in each space
println()
println("Contents")
for i in 2:map_size-1
    for j in 2:map_size-1
        print(@sprintf("%.1f |",contents[j,i]))
    end
    println()
end

# Will print how many resources are produced by a miner in each space
println()
println("Production")
for i in 2:map_size-1
    for j in 2:map_size-1
        print(@sprintf("%.1f |",prod[j,i]))
    end
    println()
end

# Will print the number of resources passing through each space
println()
println("Passthrough")
for i in 2:map_size-1
    for j in 2:map_size-1
        print(@sprintf("%.1f |",passthrough[j,i]))
    end
    println()
end

# Will print the distribution of resources in each space
println()
println("Resource Distribution")
for i in 2:map_size-1
    for j in 2:map_size-1
        print(@sprintf("%.1f |",resources[j,i]))
    end
    println()
end

Academic license - for non-commercial use only
Calculation Complete

Assignment
mr  | cd  | ml  | .   | 
mr  | cd  | md  | md  | 
mr  | cr  | h   | cl  | 
.   | mu  | mu  | mu  | 

Contents
0.0 |0.0 |0.0 |0.0 |
0.0 |0.0 |0.0 |0.0 |
0.0 |0.0 |9.0 |0.0 |
0.0 |0.0 |0.0 |0.0 |

Production
1.0 |0.0 |1.0 |0.0 |
1.0 |0.0 |1.0 |1.0 |
1.0 |0.0 |0.0 |0.0 |
0.0 |1.0 |1.0 |1.0 |

Passthrough
0.0 |2.0 |0.0 |0.0 |
0.0 |3.0 |0.0 |0.0 |
0.0 |5.0 |0.0 |2.0 |
0.0 |0.0 |0.0 |0.0 |

Resource Distribution
1.0 |1.0 |1.0 |1.0 |
1.0 |1.0 |1.0 |1.0 |
1.0 |1.0 |1.0 |1.0 |
1.0 |1.0 |1.0 |1.0 |


## 4. Results and discussion ##

### 4.A. Mineral Rich Mining
Our first test case involves a field with uniform resources and unlimited mining buildings and conveyors.  In the game Factorio this is often the case when you are starting out and only want to build a small mining operation within a larger patch of resources.  This is a basic case which demonstrates the optimal placement.

#### 4.A.1 A Basic Case
For this experiment we used a 3x3 grid in which all miners produced 1 resource regardless of where they were placed.  The parameters for this where:

```
map_size = 3
miner_speed = 1
conveyor_speed = 6
max_chest_contents = 100
number_of_chests = 1

resource_noise = 0
resource_scale = 63000

x_standard_deviation = 100
y_standard_deviation = 100

x_offset = 0
y_offset = 0
```

The resulting placement was:
 
 
||||
|---|---|---|
|md  | md  | md  | 
|cr  | cr  | h   | 
|mu  | mu  | mu  |

In this result, the chest is placed in the center with miners along the top and bottom.  In this way, each miner ("md" or "mu") drops its resources into the center row.  By having a conveyor belt ("cr") facing the chest in the remaining two spots all 6 collected resources reach the chest.

#### 4.A.2 Geometric Limitations
If we expand this to a 4x4 grid we see a more complex pattern because there is no longer a center spot for the chest to occupy.

```
map_size = 4
miner_speed = 1
conveyor_speed = 6
```

|||||
|---|---|---|---|
|mr  | cd  | ml  | .   | 
|mr  | cd  | md  | md  | 
|mr  | cr  | h   | cl  | 
|.   | mu  | mu  | mu  |

This placement is interesting because it shows that even though more resources are available (since every space has 1 resource), there is no way to get them to the chest.  This results in two unused spaces in the corners simply because there is no physical way to get resources from those corners to the chest. Ultimately the issue is that putting a miner in those corners would require another conveyor, which would net no additional output.

#### 4.A.3 Overloaded Conveyor Belts
Contrast the previous scenario with the 7x7 case which reaches 24 resources feeding into the chest and results in many unoccupied spaces even though resources are abundant and could be mined.  In this case we actually feed 6 resources into the chest from each direction, which is the maximum a conveyor can carry.  Hence there is no physical way to get more resources into the chest.

```
map_size = 7
miner_speed = 1
conveyor_speed = 6
```
||||||||
|---| --- |--- |--- |--- |--- |
|.|md|mr|cd|ml|md|.|
|mr|cd|mr|cd|ml|cd|ml|
|mr|cd|mr|cd|ml|cd|ml|
|mr|cr|cr|h|cl|cl|ml|
|.|mu|mu|cu|ml|cu|ml|
|mr|cr|cr|cu|ml|mu|.|
|.|mu|mu|mu|.|.|.|

If instead we increase the conveyor belt speed to a higher number, more mining machines can be supported since the bottleneck feeding into the chest is loosened.

#### 4.A.4 General Efficiency of Space
Ultimately the geometric limitations and conveyor speed has a considerable impact on the optimal output of the mining operation.  Below we show how the total resources collected grows with the size of the grid.

![image1.png](https://image.ibb.co/mSNWJ9/prodvsspaces.png)

In this plot you can see that the production increases proportional to the number of spaces.  This is interesting because the tradeoff is relatively linear with the size of the grid.  Here we see that an extra 2 spaces produces approximately one extra resource.  When looking at an actual grid however, two miners can efficiently feed into a single conveyor.  Hence we originally expected this scaling to be much closer to 2/3 rather than 1/2.  It's interesting that the other factors such as geometric limitations cut into the production notably.

We also investigated the total run time for the solution as shown below.  Here we tested the same grid sizes as above.

![image2.png](https://image.ibb.co/fcCOy9/runtime.png)

Here we see a very interesting trend in that larger grids evaluate considerably faster than smaller ones.  We suspect that this is because the solver can easily find a single solution that maximizes the production and from there choose a configuration that minimizes the buildings required. For a smaller grid these two parts of the objective are much more correlated because changing a building to minimize cost will almost certainly affect the output.

#### 4.A.5 Multiple Chests
Since our model is extremely flexible, we can extend it to test the effect of allowing multiple chests on the map.  With only a single chest, a 7x7 grid cannot fill the entire map since the conveyor belts feeding into the chest are already at capacity.  With two chests a 7x7 grid can be completely filled.  This allows filling the last empty spots on the map.

```
map_size = 7
miner_speed = 1
conveyor_speed = 6
max_chest_contents = 100
number_of_chests = 1
```
||||||||
|---|---|---|---|---|---|---|
|.   | md  | mr  | cd  | ml  | md  | .   | 
|mr  | cd  | mr  | cd  | ml  | cd  | ml  | 
|mr  | cd  | mr  | cd  | ml  | cd  | ml  | 
|mr  | cr  | cr  | h   | cl  | cl  | ml  | 
|.   | mu  | mu  | cu  | ml  | cu  | ml  | 
|mr  | cr  | cr  | cu  | ml  | mu  | .   | 
|.   | mu  | mu  | mu  | .   | .   | .   | 

```
map_size = 7
miner_speed = 1
conveyor_speed = 6
max_chest_contents = 100
number_of_chests = 2
```
||||||||
|---|---|---|---|---|---|---|
|md  | md  | md  | md  | mr  | cd  | ml  | 
|cr  | cr  | cr  | cr  | cr  | h   | ml  | 
|mu  | md  | md  | mu  | mr  | cu  | ml  | 
|mr  | cr  | cd  | ml  | mr  | cu  | ml  | 
|md  | md  | cd  | ml  | md  | mu  | md  | 
|cr  | cr  | h   | cl  | cl  | cl  | cl  | 
|mu  | mu  | mu  | mu  | mu  | mu  | mu  | 

Note that this is actually extremely efficient.  There are 8 empty spots remaining in the single chest case.  Adding the second chest increases the total production by 6.  This means adding the additional chest uses these 8 spots with a 75% efficiency.  As we discussed in a previous section the scaling factor for adding more spaces is usually closer to 50%.  This means adding a second chest significantly affects the efficiency of the operation.

This can be seen qualitatively in the patterns on the map.  Note that the original case has conveyors effectively spiraling out from the chest.  This results in conveyors with only single chests on them due to space limitations. With two chests, it can better place two miners on every conveyor by using a straight line along the chest's row rather than larger spirals.

### 4.B. Implementation of true Gaussian Distribution

#### 4.B.1 Basic Case

In this scenario, we set the parameters to create a true Gaussian distribution function to set the map, setting the mean in the middle of the map, the standard deviations at 1, creating no noise, and keeping building speeds at their standard value. Our results are fairly predictable. The miners are placed in the middle two rows, flowing outward to conveyors, which all flow toward a chest at the bottom right edge of the map.

Here are the parameters used for this case:

```
map_size = 4
miner_speed = 1
conveyor_speed = 6
max_chest_contents = 100
number_of_chests = 1

resource_noise = 0
resource_scale = (map_size^2)/2

x_standard_deviation = 1
y_standard_deviation = 1

x_offset = 0
y_offset = 0
```

|||||
|---|---|---|---|
|cr  | cr  | cr  | cd  | 
|mu  | mu  | mr  | cd  | 
|md  | md  | mr  | cd  | 
|cr  | cr  | cr  | h   |

One interesting future point of analysis could be to adjust the price of certain directions of miners and conveyors. An applicable use of this could be that you are given financial incentive to build in a certain direction due to an outside factor, such as getting closer to a valuable resource (for financial gain or our necessity; for example, gold or water). However, there is plenty of analysis to be done by adjusting other parameters that simply relate to flow.

#### 4.B.2 Corner Resources

In this scenario, we push the higher values of the Gaussian distribution to the bottom right corner of the map. This causes depletion of resources that are now farther from the new mean, especially in the top left corner. The goal is always to place the miners in the most resource rich area.

Changed parameters:

```
x_offset = 1
y_offset = 1
```

|||||
|---|---|---|---|
|cr  | cr  | cr  | cd  | 
|cu  | mu  | mr  | cd  | 
|cu  | md  | mr  | h   | 
|cu  | cl  | ml  | mu  | 

This is an interesting result. The chest is placed in a resource rich place, blocking off potential resources, but this is necessary to prevent a cut-off from flow if we were to place it in the upper right corner. This scenario provides lots of potential for future analysis. With a concept of expanding that map or looking into different layouts, the mean of the Gaussian distribution has tremendous impact in where the miners are placed. Moving around the mean could make for some interesting potential analysis.

#### 4.B.3 Lower Standard Deviation

In this scenario, we set the standard devation for x to 0.2. This compresses the distribution to a very small x range, so the resources are especially prevalent in the middle of the x region of the map, spread along the y-axis. Given this layout, the proper placement of the miners is in the middle two columns.

```
x_standard_deviation = 0.2
y_standard_deviation = 1

x_offset = 0
y_offset = 0
```

|||||
|---|---|---|---|
|cd  | ml  | mr  | cd  | 
|cd  | ml  | mr  | cd  | 
|cd  | md  | mr  | cd  | 
|cr  | cr  | cr  | h   | 

Now that we have observed the impact of the offset and standard deviation values, we have an understanding of what each can do to the objective. We analyzed these factors separately to gain an understanding of their individual impact. The combining of these factors would likely be a potential implementation method for consideration in the context of the actual game of Factorio.

#### 4.B.4 Noise Implementation

In this scenario, we adjust the noise parameter. We set noise_param to 0.3, which means that as we run each entry through the Gaussian probability equation, its value is multiplied by a random percentage between 70 and 130. The nature of the parameters we have set for the Gaussian distribution is such that the middle of the map is heavily rich in resources compared to the edges. As a result, it would take a significant amount of random noise to lure the miners out of the middle. Noise is more likely to have an impact when combined with a change in other parameters. In addition, rather than creating a noise parameter in which we multiply each entry by a random percentage, it could be interesting to add noise to the probability of each entry. This would increase the likelihood of a miner being placed on an entry that would normally be an outlier with very low probability. In this scenario, we see that the bottom middle entries benefitted more from the noise parameter than the other entries surrounding the center did. As a result, there were two miners placed at the bottom. We could run this function again, and the same thing may happen on the upper, left, or right entries instead.

Here are the changes in parameters:

```
resource_noise = 0.3
resource_scale = (map_size^2)/2

x_standard_deviation = 1
y_standard_deviation = 1

```

|||||
|---|---|---|---|
|cr  | cr  | cr  | cd  | 
|cu  | mu  | mr  | cd  | 
|cu  | ml  | mr  | cd  | 
|cu  | ml  | mr  | h   | 

Here is the Resource Distribution:

|||||
|---|---|---|---|
|0.2 |0.3 |0.3 |0.1 |
|0.3 |1.0 |1.0 |0.3 |
|0.4 |0.9 |1.0 |0.3 |
|0.1 |0.5 |0.4 |0.2 |

## 5. Conclusion ##

In this project we found that optimizing the layout of a factory in Factorio was possible using Mixed Integer Programming.  We found the optimal layout of mining machines and conveyor belts in a number of situations.  More importantly, we found that in many situations there was no layout that could leverage all resources available.

Performance in our model was not very high because it was a mixed integer problem. This was necessary because a given space could not be half a mining machine and half a chest. We briefly explored using an L1 regularization to encourage this instead of integer constraints, but we weren't successful in this attempt.  Getting this regularization approach to work instead of integer constraints would help considerably when scaling problems above 100 spaces.

This problem was particularly interesting because we just scratched the surface on what we could do.  We left a lot of the factorio game unexplored.  An important part of the game is combining given resources into others.  Our model does not reach this part since it focuses only on transporting the single resource.  While multiple resources do not necessarily add a lot of cognitive complexity, they do add a great deal of computational complexity because adding a second resource would nearly double our variables and constraints.  Our current solution takes a while to run even on a 10 by 10 grid, so doubling the complexity would likely hurt performance considerably.

Another future task that would bring our model closer to the actual game is allowing buildings to take multiple spaces on the map.  In the real game more advanced buildings tend to be larger.  For instance the typical mining machine is actually a 3 by 3 building.  This would be possible in a fairly simple way by constraining what buildings can be placed next to each other and adding building types for each section of the larger building.

Our model as written also supports a lot of experiments we didn't actually run.  For instance we did not test what placement patterns with rectuangular shapes resulted in even though our model only needed small modifications to support it.  Another interesting question is what the tradeoff curve between the cost of a building and the resources it provides looks like.  This is important because in the game it limits how quickly you can expand the mining operation once it is deployed.  For instance, it may be more efficient to build a facility that pays itself off quickly rather than one that produces more but pays itself off slower due to a higher initial investment.